## ETL Jobs in Machine Learning Applications

### Data Extraction

In [12]:
# importing pandas library
import pandas as pd

In [13]:
# Load the dataset
data = pd.read_csv('train.csv')

In [14]:
# Display the first few rows of the dataset
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Data Transformation

#### Handling Missing Values

In [15]:
# Fill missing values in 'Age' with the median age
data['Age'].fillna(data['Age'].median(), inplace=True)

# Fill missing values in 'Embarked' with the most common embarkation point
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# Drop the 'Cabin' column due to a high number of missing values
data.drop(columns=['Cabin'], inplace=True)

C:\Users\aayus\AppData\Local\Temp\ipykernel_20180\1610716012.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].median(), inplace=True)
C:\Users\aayus\AppData\Local\Temp\ipykernel_20180\1610716012.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

#### Feature Engineering

In [16]:
# Extract titles from names
data['Title'] = data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())

# Create a family size feature
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

#### Normalize or scale numerical features and encode categorical features.

In [17]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Select numerical and categorical columns
numerical_cols = ['Age', 'Fare', 'FamilySize']
categorical_cols = ['Sex', 'Embarked', 'Title']

# Include 'Survived' in the original DataFrame for transformation
data = data[['Survived'] + numerical_cols + categorical_cols]

# Define the transformers
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Apply the transformations
data_transformed = preprocessor.fit_transform(data.drop(columns=['Survived']))

# Get the column names after transformation
transformed_columns = numerical_cols + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols))

# Convert the transformed data back to a DataFrame for convenience
data_transformed_df = pd.DataFrame(data_transformed.toarray(), columns=transformed_columns)

# Add the 'Survived' column back to the transformed DataFrame
data_transformed_df['Survived'] = data['Survived'].values

# Display the first few rows of the transformed DataFrame
print(data_transformed_df.head())

        Age      Fare  FamilySize  Sex_female  Sex_male  Embarked_C  \
0 -0.565736 -0.502445    0.059160         0.0       1.0         0.0   
1  0.663861  0.786845    0.059160         1.0       0.0         1.0   
2 -0.258337 -0.488854   -0.560975         1.0       0.0         0.0   
3  0.433312  0.420730    0.059160         1.0       0.0         0.0   
4  0.433312 -0.486337   -0.560975         0.0       1.0         0.0   

   Embarked_Q  Embarked_S  Title_Capt  Title_Col  ...  Title_Miss  Title_Mlle  \
0         0.0         1.0         0.0        0.0  ...         0.0         0.0   
1         0.0         0.0         0.0        0.0  ...         0.0         0.0   
2         0.0         1.0         0.0        0.0  ...         1.0         0.0   
3         0.0         1.0         0.0        0.0  ...         0.0         0.0   
4         0.0         1.0         0.0        0.0  ...         0.0         0.0   

   Title_Mme  Title_Mr  Title_Mrs  Title_Ms  Title_Rev  Title_Sir  \
0        0.0     

### Data Loading

#### Storing the clean and transformed data in a relational database.

In [18]:
from sqlalchemy import create_engine

# Create an SQLite database engine
engine = create_engine('sqlite:///titanic.db')

# Save the transformed DataFrame to a table named 'titanic_transformed'
data_transformed_df.to_sql('titanic_transformed', engine, index=False, if_exists='replace')

891

In [19]:
# Function to load data from the database
def load_data_from_db(engine):
    query = "SELECT * FROM titanic_transformed"
    data_from_db = pd.read_sql(query, engine)
    return data_from_db

# Load the data
data_loaded = load_data_from_db(engine)
print(data_loaded.head())

        Age      Fare  FamilySize  Sex_female  Sex_male  Embarked_C  \
0 -0.565736 -0.502445    0.059160         0.0       1.0         0.0   
1  0.663861  0.786845    0.059160         1.0       0.0         1.0   
2 -0.258337 -0.488854   -0.560975         1.0       0.0         0.0   
3  0.433312  0.420730    0.059160         1.0       0.0         0.0   
4  0.433312 -0.486337   -0.560975         0.0       1.0         0.0   

   Embarked_Q  Embarked_S  Title_Capt  Title_Col  ...  Title_Miss  Title_Mlle  \
0         0.0         1.0         0.0        0.0  ...         0.0         0.0   
1         0.0         0.0         0.0        0.0  ...         0.0         0.0   
2         0.0         1.0         0.0        0.0  ...         1.0         0.0   
3         0.0         1.0         0.0        0.0  ...         0.0         0.0   
4         0.0         1.0         0.0        0.0  ...         0.0         0.0   

   Title_Mme  Title_Mr  Title_Mrs  Title_Ms  Title_Rev  Title_Sir  \
0        0.0     

#### Integration with ML Pipeline

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the features and target variable
X = data_loaded.drop(columns=['Survived'])
y = data_loaded['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.82
